In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv("/kaggle/input/hackerearth-deep-learning-challenge-holidayseason/dataset/train.csv")
train

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train.Class.value_counts().plot(kind='bar')
plt.title('Labels counts')
plt.xlabel('Label')
plt.ylabel('Count')
plt.show()

In [ ]:
directory = '../input/hackerearth-deep-learning-challenge-holidayseason/dataset/train'
img_width, img_height = 299, 299

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.applications.xception import Xception, preprocess_input
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input


In [ ]:
datagen = ImageDataGenerator(preprocessing_function=preprocess_input)                                

train_datagen_flow = datagen.flow_from_dataframe(
    dataframe=train,
    directory=directory,
    x_col='Image',
    y_col='Class',
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical'
)

In [ ]:
import tensorflow
from tensorflow.keras import layers
from tensorflow.keras import Sequential

from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation, LeakyReLU

In [ ]:
pre_trained_model= InceptionResNetV2(input_shape=(img_width, img_height,3),
                              include_top=False,
                              weights='imagenet')

In [ ]:
pre_trained_model.trainable = False

In [ ]:
pre_trained_model.summary()

In [ ]:
model=Sequential()
model.add(pre_trained_model)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(512,activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(256,activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(128,activation='relu'))
model.add(layers.Dense(64,activation='relu'))
model.add(layers.Dense(32,activation='relu'))
model.add(layers.Dense(6,activation='softmax'))
model.compile(optimizer = 'adam',
             loss='categorical_crossentropy',
             metrics=['acc'])

In [ ]:
train_steps = np.ceil(train_datagen_flow.n/train_datagen_flow.batch_size)

In [ ]:
history = model.fit(train_datagen_flow,
                            epochs = 12,
                            steps_per_epoch = train_steps,
                            batch_size = 16
                             )

In [ ]:
test_df = pd.DataFrame()
test_images = os.listdir('../input/hackerearth-deep-learning-challenge-holidayseason/dataset/test')
test_df['Image']=test_images
test_df.head()

In [ ]:
test_generator = datagen.flow_from_dataframe(
    test_df,
    directory='../input/hackerearth-deep-learning-challenge-holidayseason/dataset/test',
    x_col = 'Image',
    y_col = None,
    target_size=(299,299),
    class_mode = None,
    batch_size=16,
    shuffle = False)

In [ ]:
test_steps = np.ceil(test_generator.n/test_generator.batch_size)

In [ ]:
pred = model.predict_generator(
    test_generator,
    steps = test_steps,
    verbose = 1)

In [ ]:
pred = np.argmax(pred,axis=-1)

In [ ]:
test_df['Class']=pred

In [ ]:
num_to_class = dict((y,x) for (x,y) in train_datagen_flow.class_indices.items())
num_to_class

In [ ]:
test_df['Class']=test_df['Class'].map(num_to_class)
test_df.head()

In [ ]:
test_df.to_csv('pred.csv',index=False)